# Create small difficult corpus
 Notebook to create a small, yet difficult corpus, for quick experimentation. In many cases, there are a lot of documents to search through, and indexing these documents with various methods can take a very long time. To avoid lenghty experiments, yet making sure that the corpus oen searches through is "hard", this notebook creates a small corpus (e.g. 10000 documents) that contain:
  * all goldstandard documents
  * top 100 documents per query (obtained with some method - could be the model itself, another model, or bm25)
  * other random documents to pad to the desired size

In [45]:
from docuverse.utils import open_stream, read_config_file
from docuverse.engines.search_engine import SearchEngine, SearchData
import os
import orjson
from  tqdm import tqdm_notebook as tqdm

In [67]:
data_dir="/home/raduf/sandbox2/docuverse/benchmark/ibm_search"
query_file = os.path.join(data_dir, "question_test_config.json")
corpus_file = os.path.join(data_dir, "corpus.jsonl.bz2")
query_config = read_config_file(query_file)
query_json = query_config['question_file']
goldstandard = query_config['goldstandard_file']
short_doc_output = os.path.join(data_dir, "test_corpus_21k.jsonl")

In [4]:
# config_file = os.path.join("/home/raduf/sandbox2/docuverse/experiments/unified_search/ibmsw_milvus_dense.snowlake-m.test.yaml")
config_file = os.path.join("/home/raduf/sandbox2/docuverse/experiments/unified_search/ibmsw_milvus_bm25.granite.test.yaml")
engine = SearchEngine(config_file)

In [5]:
for i, line in enumerate(open_stream(corpus_file)):
    # print(line)
    print(orjson.loads(line))
    if i==10:
        break

{'title': 'aclput Command', '_id': '1', 'text': 'aclput Command Purpose Sets the access control information of a file. Syntax aclput [ -i inAclFile ] [ -R ] [ -t acl_type ] [ -v ]FileObject Description The aclput command sets the access control information of the file object that is specified by the FileObject parameter. The command reads standard input for the access control information, unless you specify the -i flag. Note: If you are reading from standard input your entries must match the expected format of the access control information or you get an error message. Use the Ctrl-D key sequence to complete the session. Access Control List Access Control Lists form the core of protection for file system objects. Each file system object is uniquely associated with one piece of data, called ACL that defines the access rights to the object. ACL might consist of multiple Access Control Entries (ACEs), each defining one particular set of access rights for a user. Typically, ACE consists of

In [6]:
corpus = [orjson.loads(line) for line in tqdm(open_stream(corpus_file))]

/tmp/ipykernel_619333/116095110.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  corpus = [orjson.loads(line) for line in tqdm(open_stream(corpus_file))]


0it [00:00, ?it/s]

In [7]:
documents = {}
for c in corpus:
    documents[c['_id']] = c

In [8]:
documents['1']

{'title': 'aclput Command',
 '_id': '1',
 'text': 'aclput Command Purpose Sets the access control information of a file. Syntax aclput [ -i inAclFile ] [ -R ] [ -t acl_type ] [ -v ]FileObject Description The aclput command sets the access control information of the file object that is specified by the FileObject parameter. The command reads standard input for the access control information, unless you specify the -i flag. Note: If you are reading from standard input your entries must match the expected format of the access control information or you get an error message. Use the Ctrl-D key sequence to complete the session. Access Control List Access Control Lists form the core of protection for file system objects. Each file system object is uniquely associated with one piece of data, called ACL that defines the access rights to the object. ACL might consist of multiple Access Control Entries (ACEs), each defining one particular set of access rights for a user. Typically, ACE consists 

In [10]:
queries = engine.read_questions(engine.config.input_queries)

Reading benchmark/ibm_search/test-queries.jsonl:: 402it [00:00, 319641.75it/s]


In [11]:
relevant = {}
for q in queries:
    for r in q['relevant']:
        relevant[r] = 1

In [39]:
answers = engine.read_cache_file(extension=".retrieve.pkl.bz2")[0]

File /home/raduf/sandbox2/docuverse/experiments/unified_search/cache/ibmsw-full-milvus-bm25-granite-512-100-574485.retrieve.pkl.bz2 exists, read?
Reading cached search results from /home/raduf/sandbox2/docuverse/experiments/unified_search/cache/ibmsw-full-milvus-bm25-granite-512-100-574485.retrieve.pkl.bz2


In [50]:
ansdocs = relevant
for a in answers:
    for result in a:
        ansdocs[SearchData.get_orig_docid(result['id'])] = 1

In [52]:
len(ansdocs)

21247

In [56]:
list(sorted(ansdocs.keys()))[0]

'10'

In [66]:
import json
with open_stream(short_doc_output, write=True) as g:
    for docid in sorted(ansdocs.keys()):
        print(json.dumps(documents[docid]), file=g)

TypeError: memoryview: a bytes-like object is required, not 'str'